In [ ]:
import pandas as pd

In [ ]:
times = []
with open('R1_time.txt') as f:
    for line in f:
        times.append(line.split('\t')[2].rstrip('\n'))
times

In [ ]:
#コロンを除去
array=[]
array_times=[]
for i in range(len(times)):
    array.append(times[i].split(':'))
#秒に変換
for x in range(len(array)):
        array_times.append((float(array[x][0])*3600+float(array[x][1])*60+float(array[x][2])))
array_times[18]=225.33
array_times[30]=165.42
array_times[42]=105.54
array_times

In [ ]:
#脳波のデータ
R1 = pd.read_csv('R1.txt', names=['time', 'ch1', 'ch2', 'ch3', 'ch4'])
R1['move'] = 0

In [ ]:
R1

In [ ]:
R1['time'][0]

In [ ]:
len(R1)

In [ ]:
t = float(array1[0]) * 3600 + float(array1[1]) * 60 + float(array1[2])
abs(R1['time'] - t)

In [ ]:
abs(R1['time'] - t)[0]

In [ ]:
print(type(times[0]))